<a href="https://colab.research.google.com/github/MyMind2/MyMind2/blob/main/BTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NN model for predicting cryptocurrency's price

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [4]:
crypto_currency = 'BTC'
against_currency = 'USD'

start = dt.datetime(2016, 1, 1)
end = dt.datetime.now()

data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', start, end)

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [34]:
prediction_days = 60
future_day = 1

In [46]:
x_train, y_train = [], []

for x in range(prediction_days, len(scaled_data)-future_day):
  x_train.append(scaled_data[x-prediction_days:x, 0])
  y_train.append(scaled_data[x:x+future_day, 0])

In [36]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [37]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

In [38]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=future_day))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
69/69 [==============================] - 17s 89ms/step - loss: 0.0086
Epoch 2/25
69/69 [==============================] - 6s 92ms/step - loss: 0.0026
Epoch 3/25
69/69 [==============================] - 6s 91ms/step - loss: 0.0022
Epoch 4/25
69/69 [==============================] - 6s 93ms/step - loss: 0.0029
Epoch 5/25
69/69 [==============================] - 6s 91ms/step - loss: 0.0018
Epoch 6/25
69/69 [==============================] - 7s 95ms/step - loss: 0.0019
Epoch 7/25
69/69 [==============================] - 7s 100ms/step - loss: 0.0020
Epoch 8/25
69/69 [==============================] - 7s 98ms/step - loss: 0.0019
Epoch 9/25
69/69 [==============================] - 7s 101ms/step - loss: 0.0020
Epoch 10/25
69/69 [==============================] - 7s 99ms/step - loss: 0.0016
Epoch 11/25
69/69 [==============================] - 7s 102ms/step - loss: 0.0018
Epoch 12/25
69/69 [==============================] - 7s 98ms/step - loss: 0.0019
Epoch 13/25
69/69 [==============

In [39]:
model.save('./modelsave/model1')

INFO:tensorflow:Assets written to: ./modelsave/model1\assets


INFO:tensorflow:Assets written to: ./modelsave/model1\assets


# testing

In [50]:
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()

test_data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', test_start, test_end)

scaled_x_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [51]:
x_test = []

for x in range(prediction_days, len(scaled_data)-future_day):
  x_test.append(scaled_x_data[x-prediction_days:x, 0])

In [52]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [54]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [56]:
predictions.shape

(2194, 1)